# Day 20 - MS Copilot

In [1]:
from collections import deque

def parse_input(file_name):
    with open(file_name, 'r') as file:
        racetrack = [list(line.strip()) for line in file]
    return racetrack

def find_start_end_positions(racetrack):
    start_pos = end_pos = None
    for r, row in enumerate(racetrack):
        for c, cell in enumerate(row):
            if cell == 'S':
                start_pos = (r, c)
            elif cell == 'E':
                end_pos = (r, c)
    return start_pos, end_pos

def bfs_with_cheating(racetrack, start, end):
    rows, cols = len(racetrack), len(racetrack[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    visited = set()
    queue = deque([(start[0], start[1], 0, False, 0)])  # (row, col, distance, has_cheated, cheat_time)

    best_time = float('inf')
    best_cheat_savings = {}

    while queue:
        r, c, dist, has_cheated, cheat_time = queue.popleft()

        if (r, c) == end:
            best_time = min(best_time, dist)
            if has_cheated:
                savings = cheat_time
                if savings >= 100:
                    best_cheat_savings[savings] = best_cheat_savings.get(savings, 0) + 1
            continue

        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                if racetrack[nr][nc] == '.' or racetrack[nr][nc] == 'E':
                    if (nr, nc, has_cheated, cheat_time) not in visited:
                        visited.add((nr, nc, has_cheated, cheat_time))
                        queue.append((nr, nc, dist + 1, has_cheated, cheat_time))
                elif racetrack[nr][nc] == '#' and not has_cheated:
                    if (nr, nc, True, dist + 2) not in visited:
                        visited.add((nr, nc, True, dist + 2))
                        queue.append((nr, nc, dist + 1, True, dist + 2))
    return best_cheat_savings

def main():
    racetrack = parse_input('input.txt')
    start_pos, end_pos = find_start_end_positions(racetrack)
    best_cheat_savings = bfs_with_cheating(racetrack, start_pos, end_pos)
    cheats_over_100 = sum(count for savings, count in best_cheat_savings.items() if savings >= 100)
    print(f'Number of cheats that save at least 100 picoseconds: {cheats_over_100}')

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [2]:
import heapq
from collections import defaultdict

def parse_input(file_name):
    with open(file_name, 'r') as file:
        racetrack = [list(line.strip()) for line in file]
    return racetrack

def find_start_end_positions(racetrack):
    start_pos = end_pos = None
    for r, row in enumerate(racetrack):
        for c, cell in enumerate(row):
            if cell == 'S':
                start_pos = (r, c)
            elif cell == 'E':
                end_pos = (r, c)
    return start_pos, end_pos

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_with_cheating(racetrack, start, end):
    rows, cols = len(racetrack), len(racetrack[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    pq = [(0, start[0], start[1], False, 0)]  # (cost, row, col, has_cheated, cheat_time)
    visited = defaultdict(lambda: float('inf'))

    best_time = float('inf')
    best_cheat_savings = {}

    while pq:
        cost, r, c, has_cheated, cheat_time = heapq.heappop(pq)

        if (r, c) == end:
            best_time = min(best_time, cost)
            if has_cheated:
                savings = cheat_time
                if savings >= 100:
                    best_cheat_savings[savings] = best_cheat_savings.get(savings, 0) + 1
            continue

        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                next_cost = cost + 1
                if racetrack[nr][nc] == '.' or racetrack[nr][nc] == 'E':
                    if visited[(nr, nc, has_cheated)] > next_cost:
                        visited[(nr, nc, has_cheated)] = next_cost
                        heapq.heappush(pq, (next_cost + heuristic((nr, nc), end), nr, nc, has_cheated, cheat_time))
                elif racetrack[nr][nc] == '#' and not has_cheated:
                    cheat_cost = cost + 1  # Adding cost for cheating
                    if visited[(nr, nc, True)] > cheat_cost:
                        visited[(nr, nc, True)] = cheat_cost
                        heapq.heappush(pq, (cheat_cost + heuristic((nr, nc), end), nr, nc, True, cheat_cost))
    
    return best_cheat_savings

def main():
    racetrack = parse_input('input.txt')
    start_pos, end_pos = find_start_end_positions(racetrack)
    best_cheat_savings = a_star_with_cheating(racetrack, start_pos, end_pos)
    cheats_over_100 = sum(count for savings, count in best_cheat_savings.items() if savings >= 100)
    print(f'Number of cheats that save at least 100 picoseconds: {cheats_over_100}')

if __name__ == "__main__":
    main()


Number of cheats that save at least 100 picoseconds: 1


In [3]:
import heapq
from collections import deque, defaultdict

def parse_input(file_name):
    with open(file_name, 'r') as file:
        racetrack = [list(line.strip()) for line in file]
    return racetrack

def find_start_end_positions(racetrack):
    start_pos = end_pos = None
    for r, row in enumerate(racetrack):
        for c, cell in enumerate(row):
            if cell == 'S':
                start_pos = (r, c)
            elif cell == 'E':
                end_pos = (r, c)
    return start_pos, end_pos

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def find_cheats(racetrack, start, end):
    rows, cols = len(racetrack), len(racetrack[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    pq = [(0, start[0], start[1], False, 0, None, None)]  # (cost, row, col, has_cheated, cheat_start, cheat_end, cheat_time)
    visited = set()
    visited.add((start[0], start[1], False))

    best_cheat_savings = defaultdict(int)

    while pq:
        cost, r, c, has_cheated, cheat_start, cheat_end, cheat_time = heapq.heappop(pq)

        if (r, c) == end:
            if has_cheated and cheat_time >= 100:
                savings = cheat_time
                best_cheat_savings[savings] += 1
            continue

        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                if racetrack[nr][nc] == '.' or racetrack[nr][nc] == 'E':
                    if (nr, nc, has_cheated) not in visited:
                        visited.add((nr, nc, has_cheated))
                        heapq.heappush(pq, (cost + 1, nr, nc, has_cheated, cheat_start, cheat_end, cheat_time))
                elif racetrack[nr][nc] == '#' and not has_cheated:
                    # Start cheating
                    for dr2, dc2 in directions:
                        nr2, nc2 = nr + dr2, nc + dc2
                        if 0 <= nr2 < rows and 0 <= nc2 < cols and (racetrack[nr2][nc2] == '.' or racetrack[nr2][nc2] == 'E'):
                            if (nr2, nc2, True) not in visited:
                                visited.add((nr2, nc2, True))
                                heapq.heappush(pq, (cost + 1, nr2, nc2, True, (nr, nc), (nr2, nc2), cost + 1))
    return best_cheat_savings

def main():
    racetrack = parse_input('input.txt')
    start_pos, end_pos = find_start_end_positions(racetrack)
    best_cheat_savings = find_cheats(racetrack, start_pos, end_pos)
    cheats_over_100 = sum(count for savings, count in best_cheat_savings.items() if savings >= 100)
    print(f'Number of cheats that save at least 100 picoseconds: {cheats_over_100}')

if __name__ == "__main__":
    main()


Number of cheats that save at least 100 picoseconds: 0


In [4]:
from collections import deque

def parse_input(file_name):
    with open(file_name, 'r') as file:
        racetrack = [list(line.strip()) for line in file]
    return racetrack

def find_start_end_positions(racetrack):
    start_pos = end_pos = None
    for r, row in enumerate(racetrack):
        for c, cell in enumerate(row):
            if cell == 'S':
                start_pos = (r, c)
            elif cell == 'E':
                end_pos = (r, c)
    return start_pos, end_pos

def bfs_with_cheating(racetrack, start, end):
    rows, cols = len(racetrack), len(racetrack[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    queue = deque([(start[0], start[1], 0, False)])  # (row, col, distance, has_cheated)
    visited = set()
    visited.add((start[0], start[1], False))

    best_cheat_savings = []

    while queue:
        r, c, dist, has_cheated = queue.popleft()

        if (r, c) == end:
            if has_cheated and dist > 2:
                best_cheat_savings.append(dist - 2)
            continue

        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                if racetrack[nr][nc] == '.' or racetrack[nr][nc] == 'E':
                    if (nr, nc, has_cheated) not in visited:
                        visited.add((nr, nc, has_cheated))
                        queue.append((nr, nc, dist + 1, has_cheated))
                elif racetrack[nr][nc] == '#' and not has_cheated:
                    for dr2, dc2 in directions:
                        nr2, nc2 = nr + dr2, nc + dc2
                        if 0 <= nr2 < rows and 0 <= nc2 < cols and (racetrack[nr2][nc2] == '.' or racetrack[nr2][nc2] == 'E'):
                            if (nr2, nc2, True) not in visited:
                                visited.add((nr2, nc2, True))
                                queue.append((nr2, nc2, dist + 2, True))

    # Count cheats that save at least 100 picoseconds
    cheats_over_100 = sum(1 for cheat in best_cheat_savings if cheat >= 100)
    return cheats_over_100

def main():
    racetrack = parse_input('input.txt')
    start_pos, end_pos = find_start_end_positions(racetrack)
    cheats_over_100 = bfs_with_cheating(racetrack, start_pos, end_pos)
    print(f'Number of cheats that save at least 100 picoseconds: {cheats_over_100}')

if __name__ == "__main__":
    main()


Number of cheats that save at least 100 picoseconds: 0


In [5]:
from collections import deque
import sys

def parse_input(file_name):
    with open(file_name) as f:
        return [list(line.strip()) for line in f]

def find_positions(grid):
    start = end = None
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 'S':
                start = (i, j)
            if grid[i][j] == 'E':
                end = (i, j)
    return start, end

def bfs(grid, start, end):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    rows, cols = len(grid), len(grid[0])
    visited = [[False] * cols for _ in range(rows)]
    queue = deque([(start, 0, False)])  # (position, steps, has_cheated)
    visited[start[0]][start[1]] = True

    while queue:
        (x, y), steps, has_cheated = queue.popleft()

        if (x, y) == end:
            return steps

        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            if 0 <= nx < rows and 0 <= ny < cols and not visited[nx][ny]:
                if grid[nx][ny] == '.':
                    visited[nx][ny] = True
                    queue.append(((nx, ny), steps + 1, has_cheated))
                elif grid[nx][ny] == '#' and not has_cheated:
                    # Attempt cheat
                    for ddx, ddy in directions:
                        nnx, nny = nx + ddx, ny + ddy
                        if 0 <= nnx < rows and 0 <= nny < cols and grid[nnx][nny] == '.' and not visited[nnx][nny]:
                            visited[nnx][nny] = True
                            queue.append(((nnx, nny), steps + 2, True))

    return sys.maxsize  # In case there's no valid path

def solve_race(file_name):
    grid = parse_input(file_name)
    start, end = find_positions(grid)
    return bfs(grid, start, end)

file_name = 'input.txt'
min_steps = solve_race(file_name)
print(f'Minimum steps: {min_steps}')


Minimum steps: 9223372036854775807


## Part 2